## get results info

In [1]:
from pathlib import Path
import pickle

from collections import defaultdict

from utils.helpers import rtt_to_km, distance

measurement_file = Path(".") / "../datasets/test_geoloc/10_probes_10_target.pickle"
target_anchor_file = "../datasets/target_anchors.pickle"
vp_anchor_file = "../datasets/vp_anchors.pickle"


with open(target_anchor_file, "rb") as f:
    target_anchors = pickle.load(f)

with open(vp_anchor_file, "rb") as f:
    vp_anchors = pickle.load(f)
# get measurement results
with open(measurement_file, "rb") as f:
    measurement_results = pickle.load(f)

geoloc_results = defaultdict(dict)
for target_ip in measurement_results:

    print(f"results for {target_ip}, measurement_id: {measurement_results[target_ip]['measurement_id']}")

    for result in measurement_results[target_ip]['results']:

        # parse results and calculate geoloc
        if result.get('result') is not None:
            try:
                probe_ip = result['from']
                min_rtt =  min([list(rtt.values())[0] for rtt in result['result']])

                vp_lat = vp_anchors[probe_ip]['latitude']
                vp_lon = vp_anchors[probe_ip]['longitude']
                target_lat = target_anchors[target_ip]["latitude"]
                target_lon = target_anchors[target_ip]["longitude"]

                geoloc_results[target_ip][probe_ip] = {}
                geoloc_results[target_ip][probe_ip]['rtt'] = min_rtt
                geoloc_results[target_ip][probe_ip]["latitude"]= vp_lat
                geoloc_results[target_ip][probe_ip]["longitude"]= vp_lon

                geoloc_results[target_ip][probe_ip]["calculated_d"] = distance(
                    vp_lat,
                    target_lat,
                    vp_lon,
                    target_lon
                )

                geoloc_results[target_ip][probe_ip]["measured_d"] = rtt_to_km(min_rtt)

                print(
                    f"target: {result['dst_addr']}: {target_anchors[target_ip]} \n \
                    anchor: {probe_ip}: {vp_anchors[probe_ip]} \n \
                    mean rtt: {geoloc_results[target_ip][probe_ip]} \n \
                    distance (calculated): {geoloc_results[target_ip][probe_ip]['calculated_d']} \n \
                    distance (measured)  : {geoloc_results[target_ip][probe_ip]['measured_d']}"
                )

            except TypeError:
                print(f"no results: {result}")
                continue
        else:
            print(f"no results: {result}")

    #print()

results for 207.226.224.236, measurement_id: 52415122
target: 207.226.224.236: {'id': 6784, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1095, 'longitude': 8.7015} 
                     anchor: 83.243.44.55: {'id': 6413, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1385, 'longitude': 8.7375} 
                     mean rtt: {'rtt': 0.922516, 'latitude': 50.1385, 'longitude': 8.7375, 'calculated_d': 4.121290771729135, 'measured_d': 23.0629} 
                     distance (calculated): 4.121290771729135 
                     distance (measured)  : 23.0629
target: 207.226.224.236: {'id': 6784, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1095, 'longitude': 8.7015} 
                     anchor: 185.120.22.29: {'id': 6433, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1385, 'longitude': 8.7385} 
                     mean rtt: {'rtt': 1.026076, 'latitude': 50.1385, 'longitude': 8.7385, 'calculated_d': 4.166058688057477, 'measured_d': 25.6518999999999

In [2]:
from pathlib import Path
from copy import copy

from utils.helpers import polygon_centroid, haversine, circle_intersections
from utils.draw_results import draw_results

measurement_file = Path(".") / "../databases/test/10_probes_10_target.pickle"
probes_path = Path(".") / "../datasets/probe_atlas.pickle"

def select_best_guess_centroid(target_ip, measurement_results):
    """
    Find the best guess
    that is the location of the vantage point closest to the centroid.
    """
    probe_circles = {}
    for _, probe_ip in enumerate(measurement_results):
        if measurement_results[probe_ip]:
            lat = measurement_results[probe_ip]["latitude"]
            lon = measurement_results[probe_ip]["longitude"]
            rtt = measurement_results[probe_ip]["rtt"]

            # too inflated RTT means that measurement will not provide usefull info
            if isinstance(rtt, float):
                if rtt < 100:
                    probe_circles[probe_ip] = (
                        lat,
                        lon,
                        rtt,
                        None,
                        None,
                    )
                    print(f"target ip {target_ip}")
                    print(f"vp_anchor = {vp_anchors[probe_ip]}")
                    print(f"probe_circle = {lat, lon,rtt}")
            else:
                # TODO: remove measurements were not RTT is available
                print(f"measurement rtt is not int : {rtt}")

    # draw circles
    if not probe_circles: return

    intersections = circle_intersections(probe_circles.values(), speed_threshold=4/9)
    print("intersecion:", intersections)
    target = (target_anchors[target_ip]["latitude"], target_anchors[target_ip]["longitude"])

    draw_results(copy(probe_circles), intersections, target)

    for ip, circle in probe_circles.items():
        print(f"target ip {ip}: results : {circle}")

    if intersections:
        centroid = polygon_centroid(intersections)
        print("centroid:", centroid)
        target_description = target_anchors[target_ip]

        if centroid:
            for ip_src, circle in probe_circles.items():

                measured_d = haversine((circle[0], circle[1]), centroid)
                calculated_d = haversine((target_description["latitude"], target_description["longitude"]), centroid)

                print("ip_src:",ip_src,"dst (measured):", measured_d, "dst (calculated):", calculated_d)

for i, (target_ip, geoloc_result) in enumerate(geoloc_results.items()):
    select_best_guess_centroid(target_ip, geoloc_result)

target ip 207.226.224.236
vp_anchor = {'id': 6413, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1385, 'longitude': 8.7375}
probe_circle = (50.1385, 8.7375, 0.922516)
target ip 207.226.224.236
vp_anchor = {'id': 6433, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1385, 'longitude': 8.7385}
probe_circle = (50.1385, 8.7385, 1.026076)
target ip 207.226.224.236
vp_anchor = {'id': 6558, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1115, 'longitude': 8.7395}
probe_circle = (50.1115, 8.7395, 1.086811)
target ip 207.226.224.236
vp_anchor = {'id': 6561, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1175, 'longitude': 8.7405}
probe_circle = (50.1175, 8.7405, 0.59629)
target ip 207.226.224.236
vp_anchor = {'id': 6601, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1375, 'longitude': 8.7375}
probe_circle = (50.1375, 8.7375, 0.795195)
target ip 207.226.224.236
vp_anchor = {'id': 6708, 'is_anchor': True, 'country_code': 'DE', 'latitude': 50.1115, 'lo

target ip 83.243.44.55: results : (50.1385, 8.7375, 0.922516, None, None)
target ip 185.120.22.29: results : (50.1385, 8.7385, 1.026076, None, None)
target ip 85.197.82.113: results : (50.1115, 8.7395, 1.086811, None, None)
target ip 45.76.86.163: results : (50.1175, 8.7405, 0.59629, None, None)
target ip 194.50.92.201: results : (50.1375, 8.7375, 0.795195, None, None)
target ip 34.89.240.90: results : (50.1115, 8.6775, 1.15306, None, None)
target ip 217.243.20.18: results : (50.1095, 8.6785, 91.001828, None, None)
target ip 81.91.160.93: results : (50.1205, 8.7395, 6.881116, None, None)
target ip 139.162.139.168: results : (50.0975, 8.6395, 0.82642, None, None)
target ip 46.101.130.201: results : (50.1205, 8.7405, 0.697606, None, None)
target ip 156.154.76.254: results : (50.1215, 8.7405, 0.660892, None, None)
target ip 141.98.136.85: results : (50.1405, 8.7415, 1.172781, None, None)
target ip 194.145.125.9: results : (50.1215, 8.7405, 1.25854, None, None)
target ip 185.32.187.206: re

target ip 83.243.44.55: results : (50.1385, 8.7375, 1.596014, None, None)
target ip 185.120.22.29: results : (50.1385, 8.7385, 1.763706, None, None)
target ip 2.58.52.152: results : (50.1095, 8.6785, 1.989033, None, None)
target ip 85.197.82.113: results : (50.1115, 8.7395, 1.616209, None, None)
target ip 45.76.86.163: results : (50.1175, 8.7405, 1.942369, None, None)
target ip 194.50.92.201: results : (50.1375, 8.7375, 1.895029, None, None)
target ip 34.89.240.90: results : (50.1115, 8.6775, 2.091506, None, None)
target ip 217.243.20.18: results : (50.1095, 8.6785, 0.908223, None, None)
target ip 81.91.160.93: results : (50.1205, 8.7395, 7.453614, None, None)
target ip 139.162.139.168: results : (50.0975, 8.6395, 1.21047, None, None)
target ip 46.101.130.201: results : (50.1205, 8.7405, 1.28169, None, None)
target ip 156.154.76.254: results : (50.1215, 8.7405, 1.423144, None, None)
target ip 141.98.136.85: results : (50.1405, 8.7415, 1.384378, None, None)
target ip 194.145.125.9: resu

target ip 197.80.104.36: results : (-26.1415, 28.0095, 60.244426, None, None)
target ip 196.49.5.98: results : (-6.8105, 39.2885, 13.350908, None, None)
target ip 188.172.217.86: results : (-26.1385, 28.2005, 51.59246, None, None)
target ip 41.221.208.94: results : (-25.9795, 32.5805, 65.591572, None, None)
target ip 156.154.135.254: results : (-26.1415, 28.2015, 61.718199, None, None)
target ip 197.189.194.147: results : (-25.9315, 28.1405, 60.865984, None, None)
target ip 130.59.80.2
vp_anchor = {'id': 7131, 'is_anchor': True, 'country_code': 'CH', 'latitude': 47.4975, 'longitude': 8.7185}
probe_circle = (47.4975, 8.7185, 4.538391)
Not enough circles (0).
intersecion: []
calculated intersections:
target:
47.4975 8.7185


target ip 77.109.180.62: results : (47.4975, 8.7185, 4.538391, None, None)
target ip 77.88.216.1
vp_anchor = {'id': 6453, 'is_anchor': True, 'country_code': 'RO', 'latitude': 47.1585, 'longitude': 27.5895}
probe_circle = (47.1585, 27.5895, 51.458383)
measurement rtt is not int : *
target ip 77.88.216.1
vp_anchor = {'id': 6845, 'is_anchor': True, 'country_code': 'BG', 'latitude': 44.1195, 'longitude': 27.2595}
probe_circle = (44.1195, 27.2595, 51.457313)
target ip 77.88.216.1
vp_anchor = {'id': 7145, 'is_anchor': True, 'country_code': 'RO', 'latitude': 44.4795, 'longitude': 26.1195}
probe_circle = (44.4795, 26.1195, 63.734817)
Not enough circles (2).
intersecion: []
calculated intersections:
target:
44.4795 26.1195


target ip 86.107.73.26: results : (47.1585, 27.5895, 51.458383, None, None)
target ip 195.250.60.125: results : (44.1195, 27.2595, 51.457313, None, None)
target ip 185.86.67.69: results : (44.4795, 26.1195, 63.734817, None, None)
